In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize
from modules.KNN import *
from modules.stats import * 
from modules.PCA import * 
from modules.LDA import * 
from modules.read_data import *
from modules.ROC import *

In [ ]:
path = '../../Assets/Synthetic Dataset'
X, Y, X_d, Y_d = read_SD(path)

In [ ]:
pca = False
lda = False
lda = ~pca & lda #if pca is true then lda is false

In [ ]:
if pca:
	l = 1
	train_mean, train_cov = stats(X)
	eigval, Q = PCA(train_cov, l)
	X = X @ Q
	X_d = X_d @ Q
if lda:
	pc = 1
	μ, Σ = stats(X)
	μk = []
	for c in range(2):
		μk.append(stats(X[Y == c])[0])
	μk = np.array(μk)
	v = LDA(X,Y,μ,μk,pc)
	X = np.abs(X @ v)
	X_d = np.abs(X_d @ v)

In [ ]:
C1 = X[Y== 0]
C2 = X[Y == 1]
D1 = X_d[Y_d == 0]
D2 = X_d[Y_d == 1]

In [ ]:
C = []
D = []
C.append(C1)
C.append(C2)
D.append(D1)
D.append(D2)

In [ ]:
##KNN implementation
total_train = X
dev_data = D
kn = 15

prediction = []
P = []
for c in range(2):
	p = np.zeros(dev_data[c].shape[0])
	a = np.zeros((dev_data[c].shape[0], 2))
	for i in range(dev_data[c].shape[0]):
		p[i] = KNN(dev_data[c][i], total_train, Y, kn)[0]
		A = KNN(dev_data[c][i], total_train, Y, kn)[1]
		a[i] = np.array([(A==0).sum()/kn, (A==1).sum()/kn])
	prediction.append(p)
	P.append(a)
P = np.concatenate(P)

In [ ]:
cm = confusion_matrix(np.array(prediction).flatten(), Y_d)
accuracy = accuracy_score(np.array(prediction).flatten(),Y_d)
ax = sns.heatmap(cm, annot = True, cmap ='plasma',linecolor ='black', linewidths = 1, fmt = '.0f', xticklabels = '12', yticklabels = '12')
ax.set_title(f'Accuracy = {accuracy}')

In [ ]:
TPR, FPR = ROC(Y_d, P[:,1])
plt.plot(FPR, TPR)
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title("ROC")